# Vessels API Example

## Run this example in [Colab](https://colab.research.google.com/github/SignalOceanSdk/SignalSDK/blob/master/docs/examples/jupyter/VesselsAPI/VesselsAPI.ipynb) 

<h3>Setup</h3>

Install the Signal Ocean SDK:

In [ ]:
pip install signal-ocean

Import required libraries / APIs / packages:

<h3>Import Vessels API and some additional libraries</h3>

In [1]:
from signal_ocean.vessels import VesselsAPI
import pandas as pd
from datetime import datetime
from signal_ocean import Connection

Fill in your personal SignalOcean API subscription key acquired <a href="https://apis.signalocean.com/profile" target="_blank">here</a> and open a connection:

In [2]:
signal_ocean_api_key = '' #replace with your subscription key

Create a connection

In [3]:
connection = Connection(signal_ocean_api_key)

## Call the vessels API
The Vessels API retrieves vessel information.

In [4]:
api = VesselsAPI(connection)

#### Get vessel classes

Retrieve information on the various vessel classes

In [5]:
vessel_classes = api.get_vessel_classes()

Extract the returned information into a dataframe

In [6]:
pd.DataFrame([vc.__dict__ for vc in vessel_classes]).head()

,id,vessel_type_id,from_size,to_size,name,vessel_type,defining_size,size
0,-2,-2,0,0,Not set,Not Set,NotSet,NotSet
1,-1,-1,0,0,Unknown,Unknown,Unknown,Unknown
2,60,6,70001,250000,VLGC,LPG,CubicSize,cbm
3,61,6,28001,70000,Midsize/LGC,LPG,CubicSize,cbm
4,62,6,10001,28000,Handy,LPG,CubicSize,cbm


#### Get vessel types

Retrieve information on the various vessel types

In [7]:
vessel_types = api.get_vessel_types()

Extract the returned information into a dataframe

In [8]:
pd.DataFrame([vc.__dict__ for vc in vessel_types]).head(7)

,id,name
0,-2,Not Set
1,-1,Unknown
2,6,LPG
3,3,Dry
4,4,Container
5,1,Tanker
6,5,LNG


#### Get details for a specific vessel

In [9]:
imo = 9436006
v = api.get_vessel(imo)
print(f'{v.imo}: {v.vessel_name} ({v.vessel_class} / {v.commercial_operator})')

9436006: Samsun (Aframax / Unknown)


#### Get details for all vessels

In [10]:
vessels = api.get_vessels()

Extract the returned information into a dataframe

In [11]:
df = pd.DataFrame([x.__dict__ for x in vessels])

In [12]:
df[['imo', 'vessel_name', 'vessel_class']].sample(5)

,imo,vessel_name,vessel_class
56036,9936264,Seacon Nola,Post Panamax
56348,9941398,Nord Aquarius,Panamax
30758,9371311,Santorini,Small
5338,8014746,Akti N.,MR2
49241,9766231,Seacon Manila,Handysize


#### Find fleet size per vessel class - exclude scrapped vessels

In [13]:
df[pd.isnull(df['scrapped_date'])]['vessel_class'].value_counts().to_frame('vessel_count').head(10)

,vessel_count
vessel_class,
Small,21340
Panamax,4164
Supramax,3685
Handysize,3358
MR2,1946
Post Panamax,1815
Capesize,1687
Feeder,1590
Aframax,1304


#### Find the commercial operators that currently operate the largest Aframax fleets

In [14]:
data = df[(df['vessel_class']=='Aframax')&(pd.isnull(df['scrapped_date']))]
data['commercial_operator'].value_counts().head(10)

commercial_operator
Unknown                          208
Sovcomflot                        46
Trafigura                         35
Cardiff Marine                    31
Scorpio Commercial Management     30
Teekay Corp                       29
Penfield Marine                   28
Shell                             28
Minerva Marine                    27
Navig8 group                      23
Name: count, dtype: int64

#### Get all vessels the name of which contains the term helen

In [15]:
vessels = api.get_vessels('helen')
len(vessels)

40

Extract the returned information into a dataframe

In [16]:
df = pd.DataFrame([x.__dict__ for x in vessels])
df.head()

,imo,vessel_type_id,built_for_trade_id,trade_id,vessel_class_id,commercial_operator_id,deadweight,breadth_extreme,gross_rated_tonnage,reduced_gross_tonnage,...,bwts,grabs_fitted,ghg,order_book_status_id,order_book_status,order_date,construction_start_date,launch_date,scheduled_delivery_date,cancelled_date
0,6716857,4,-2,-1,95,-1,1180,11,953,762,...,None,None,None,8,Dead,NaT,NaT,NaT,NaT,None
1,7360083,1,1,1,84,-1,274333,52,133871,98338,...,None,None,None,8,Dead,NaT,1976-01-01 00:00:00+00:00,1976-01-01 00:00:00+00:00,NaT,None
2,7915644,3,4,4,74,-1,70537,32,41644,33659,...,None,None,None,8,Dead,1979-05-01 00:00:00+00:00,NaT,NaT,NaT,None
3,8120571,3,4,4,77,-1,34913,28,20922,16169,...,None,None,None,8,Dead,1981-11-01 00:00:00+00:00,1982-02-15 00:00:00+00:00,1982-09-16 00:00:00+00:00,NaT,None
4,8716306,3,5,5,92,-1,3130,19,6767,4205,...,None,None,None,7,Live,NaT,1990-01-01 00:00:00+00:00,1990-01-01 00:00:00+00:00,NaT,None


#### Get Vessel Names Changes

You can get Vessel Name changes for all vessels or for a specific one 

In [17]:
imo = 9436006
vessels_names = api.get_vessels_name_history(imo)

In [18]:
df = pd.DataFrame([x.__dict__ for x in vessels_names[0].history[0].values])

In [19]:
df

,value,begin_date,end_date,name
0,Stealth Chios,1900-01-01 00:00:00+00:00,2012-01-01 23:13:55+00:00,None
1,Signal Cheetah,2012-01-01 23:13:55+00:00,2021-02-25 23:56:31+00:00,None
2,Stealth Chios,2021-02-25 23:56:31+00:00,2021-05-01 07:41:09+00:00,None
3,Paramount,2021-05-01 07:41:09+00:00,2023-03-10 18:59:11+00:00,None
4,Samsun,2023-03-10 18:59:11+00:00,9999-12-31 23:59:59.999999+00:00,None


#### Get Commercial Operator Changes

You can get Vessel Name changes for all vessels or for a specific one 

In [20]:
imo = 9436006
vessels_names = api.get_vessels_commOp_history(imo)

In [21]:
df = pd.DataFrame([x.__dict__ for x in vessels_names[0].history[0].values])

In [22]:
df

,value,begin_date,end_date,name
0,1926,1900-01-01 00:00:00+00:00,2016-06-12 13:03:12+00:00,Signal Maritime
1,1713,2016-06-12 13:03:12+00:00,2016-06-13 13:03:12+00:00,Trafigura
2,1926,2016-06-13 13:03:12+00:00,2021-02-22 00:00:00+00:00,Signal Maritime
3,1597,2021-02-22 00:00:00+00:00,2023-04-06 15:02:40.715274+00:00,Stealth Maritime Corp
4,-1,2023-04-06 15:02:40.715274+00:00,9999-12-31 23:59:59.999999+00:00,Unknown
